In [ ]:
%matplotlib inline

import torch    as t
import torch.nn as nn

from torch.autograd import Variable

import numpy as np
import matplotlib.pyplot as plt
from itertools import islice

# TP Reseaux de neurones et Convolution

Dans ce TP, vous utilisez le formalisme le plus haut niveau de pyTorch (très proche de Keras), qui masque tous les détails

# Partie 0 - fonction de visualisation (a ne pas lire)

In [ ]:
def visualize_pytorch_classifier(X, y, predict=None,**kwargs):
    X_ = X.data.numpy()
    y_ = y.data.numpy()
    
    ax = plt.gca()
    
    # Plot the training points
    ax.scatter(X_[:, 0], X_[:, 1], c=y_, s=30, cmap='rainbow',
               clim=(y_.min(), y_.max()), zorder=3)
    ax.axis('tight')
    #ax.axis('off')
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()

    if predict:
        xx, yy = np.meshgrid(np.linspace(*xlim, num=200),
                             np.linspace(*ylim, num=200))
        xxyy   = np.c_[xx.ravel(), yy.ravel()]
        Z      = np.array([predict(Variable(t.from_numpy(d)).float(),**kwargs).data.numpy()
                           for d in xxyy]).reshape(xx.shape)

        # Create a color plot with the results
        n_classes = len(np.unique(y.data.numpy()))
        contours = ax.contourf(xx, yy, Z, alpha=0.3,
                               levels=np.arange(n_classes + 1) - 0.5,
                               cmap='rainbow', #clim=(y_.min(), y_.max()),
                               zorder=1)

        ax.set(xlim=xlim, ylim=ylim)


# Partie 1 - Régression Logistique

In [ ]:
# On reprend les memes donnees que dans le TP précédent
# On va apprendre une régression logistique dessus
n = 400

# On crée des donnees non separables lineairement:
X = Variable(t.randn(n,2))
y = X[:,0]**2-2*X[:,1]>0
y = y.float()

visualize_pytorch_classifier(X,y)

In [ ]:
net = nn.Sequential(
    nn.Linear(2, 1),   # Couche lineaire z = w0+w1.x1+x2.x2.   Les parametres wj sont initialises aleatoirement
    nn.Sigmoid()       # fonction de transfert sigmoide
)

fonction_de_cout = t.nn.BCELoss()  # fonction de cout de la regression logistique: y.log(p)+(1-y).log(1-p)

In [ ]:
# prediction sur l'exemple 0:
y_pred = net(X[0])
print( y_pred )

# cout
print( fonction_de_cout( y_pred,y[0]) )

In [ ]:
# Tous les parametres sont stockes dans net.parameters
print(list( net.parameters() ))

In [ ]:
def simple_learn(X,y,net,fonction_de_cout,ntrials=500,learning_rate = 0.05):

    for trial in range(ntrials):
        i = np.random.randint(0,n)
        xi= X[i]

        y_pred = net(xi)
        cout   = fonction_de_cout(y_pred, y[i])

        cout.backward()

        for param in net.parameters():
            param.data -= learning_rate * param.grad.data

        net.zero_grad()

In [ ]:
simple_learn(X,y,net,fonction_de_cout,ntrials=500,learning_rate = 0.05)
visualize_pytorch_classifier(X,y,net)

# Partie 2 - Réseau de neurones multi-couches

Ici, on va recoder avec `nn.Sequential` le réseau de neurones du TP précédent... mais en beaucoup moins de lignes de code !

In [ ]:
# On crée le meme réseau (a 2 couches cachées) que durant le dernier TP
net2 = nn.Sequential(
    nn.Linear(2, 3),
    nn.Sigmoid(),
    nn.Linear(3,1),
    nn.Sigmoid()
)

# et on applique la meme procédure d'apprentissage
simple_learn(X,y,net2,fonction_de_cout,ntrials=1000,learning_rate = 0.5)

# et on visualise
visualize_pytorch_classifier(X,y,net2)

#### Q: Affichez les poids des différents neurones
#### Q: Construisez un réseau à 3 couches cachées et deux neurones par couche cachée, et entrainez-le de nouveau
#### Q: Créez une base de test `X_test` et `y_test` en utilisant la même procédure de création de données que pour `X` et `y`, et mesurez l'erreur de classification comise par ces réseaux sur ce jeu de test

# Partie 3 - Réseau de neurones multi-couches appliqué aux données MNIST

Le code ci-dessous (il n'est pas utile de le lire) charge 2000 images de la base d'images de chiffres MNIST.

Les 1000 premieres sont stockees dans `X[i]`, et les 1000 suivantes dans `X_test[i]` sous forme de tableau `28x28`.

De meme, `y[i]` et `y_test[i]` identifient le chiffre correspondant à l'images `X[i]` et   `X_test[i]`

In [ ]:

from torch.utils.data.dataloader import DataLoader
from torchvision import datasets
from torchvision import transforms


mnist = datasets.MNIST(root='./data/',
                       train=True,
                       transform=transforms.ToTensor(),
                       download=True)

mnist_data_loader = t.utils.data.DataLoader(dataset=mnist,
                                            batch_size=1000, 
                                            shuffle=True)

mnist_data = next(iter(mnist_data_loader))
X,y = Variable(mnist_data[0].squeeze()),Variable(mnist_data[1])

mnist_data = next(iter(mnist_data_loader))
X_test,y_test = Variable(mnist_data[0].squeeze()),Variable(mnist_data[1])

In [ ]:
print("classe = ",y.data[0])
plt.imshow(X.data[0]);

#### Q les images dans `X` sont des matrices 28x28. En entrée du réseau, vous devrez avoir ici des vecteurs et non des matrices, pour chaque exemple. Il faut donc "applatir" ces matrices en vecteurs. En numpy, cela se fait avec `reshape` et en pytorch avec `view`, qui s'utilise à peu près comme `reshape`. Appliquez `view` sur `X` pour que `X` devienne une matrice ou chaque ligne représente une image de 784 pixels (c'est à dire 28*28)


#### Q: Telle qu'on l'a vu jusqu'à présent, la régression logistique permet de traiter des problèmes d'apprentissage à deux classes.  On cherchera donc à prédire si l'image d'un nombre représente un chiffre supérieur ou égal à 5, ou non. Reformulez `y` de telle sorte que `y[i]` soit égal à 1 si le chiffre est supérieu ou égal à 5, et 0 sinon

#### Q: Appliquez la régression logistique du début du TP sur cette tâche, sur les images X.

#### Q: Calculez le taux d'erreur du modèle obtenu sur X_test

#### Q: recommencez avec un réseau de neurones plus complexe que précédemment.

#### Q: Insérez une couche de "dropout" dans le modèle (http://pytorch.org/docs/master/nn.html#dropout)

# Partie 3 - Réseau de Convolution appliqués à MNIST

#### Q: On n'applique pas des convolutions sur des vecteurs, mais sur des tenseurs. Transormez `X` pour que sa nouvelle dimension soit 1000x1x28x28  (c'est à dire 1000 images, 1 canal, largeur 28, hauteur 28)

On va maintenant créer un réseau de convolution.

#### Q: construisez un réseau comportant une couche de convolution 3x3 avec un seul canal, suivie d'une fonction d'activation ReLU. Les fonctions à utiliser sont `nn.Conv2d(in_channels, out_channels, kernel_size)` et `nn.ReLU`.

#### Q: Si vous appliquez votre réseau sur une image, qu'obtenez-vous ? Affichez le résultat comme une image. Faites bien attention aux dimension.

#### Q: maintenant, écrivez deux modèles Sequential: un réseau qui prend un tenseur d'image en entrée, et qui fait une couche de convolutions et renvoie un tenseur d'image en sortie. Un autre qui crée un applique une régression logistique sur un vecteur de taille 784. Ecrivez une fonction `predict` qui calcule le résultat de l'application successive de ces deux réseaux sur un tenseur image.